# QKDシミュレータを用いたHTTPS通信

#### 事前準備

- [QuKayDee > Getting Started](https://qukaydee.com/pages/getting_started)を参考に、KME-1,2とSAE-1,2を登録してください。
- 作成した証明書などは、`certs`ディレクトリに保存してください。

## [STEP1] QKDシミュレータを用いた共通鍵作成

In [1]:
from entity.key_management_entity import KeyManagementEntity

In [ ]:
kme_1 = KeyManagementEntity(
        kme_id="kme-1",
        account_id="<YOUR ACCOUNT ID>",  # あなたのアカウントIDを埋めてください
        ca_cert="certs/account-<YOUR ACCOUNT ID>-server-ca-qukaydee-com.crt",
        client_cert=("certs/sae-1.crt", "certs/sae-1.key"),
    )

kme_2 = KeyManagementEntity(
        kme_id="kme-2",
        account_id="<YOUR ACCOUNT ID>",
        ca_cert="certs/account-<YOUR ACCOUNT ID>-server-ca-qukaydee-com.crt",
        client_cert=("certs/sae-2.crt", "certs/sae-2.key"),
    )

### SAE-1側の作業

In [3]:
print("--- SAE-1がSAE-2と共有する鍵を要求 ---")
status_code, keys = kme_1.get_key("sae-2", 1, 1024)
print(f"Status: {status_code}")

key_data = keys["keys"][0]
key_id_from_kme1 = key_data["key_ID"]
key_from_kme_1 = key_data["key"]
print(f"Key ID: {key_id_from_kme1}")
print(f"Key: {key_from_kme_1}")

--- SAE-1がSAE-2と共有する鍵を要求 ---
Status: 200
Key ID: 21982c03-93aa-4024-bfdf-02bedf7ffd1e
Key: P3lIOPwZ8AuffrkiayFdESHb64Iv1KgiXQKkgELbw4XMCpOmRW4kTNNC1xTm4hebfI9CYcXZwrh6xMoJxv+r4XgcBunO5RA79sFr1I2NM0KqIW8pRAmaivj6Gk94dV+ruwreTYEKEUL/yGtMb15GtuA/V4YbHAQlHlotcNHwXT4=


### SAE-2側の作業

In [4]:
print("\n--- SAE-2がSAE-1と共有した鍵を取得 ---")
status_code, keys = kme_2.get_key_with_id("sae-1", key_id_from_kme1)
print(f"Status: {status_code}")

key_data = keys["keys"][0]
key_from_kme_2 = key_data["key"]
print(f"Key: {key_from_kme_2}")


--- SAE-2がSAE-1と共有した鍵を取得 ---
Status: 200
Key: P3lIOPwZ8AuffrkiayFdESHb64Iv1KgiXQKkgELbw4XMCpOmRW4kTNNC1xTm4hebfI9CYcXZwrh6xMoJxv+r4XgcBunO5RA79sFr1I2NM0KqIW8pRAmaivj6Gk94dV+ruwreTYEKEUL/yGtMb15GtuA/V4YbHAQlHlotcNHwXT4=


In [5]:
print("\n--- 取得した鍵が一致するか確認 ---")
assert key_from_kme_1 == key_from_kme_2
print("OK")


--- 取得した鍵が一致するか確認 ---
OK


## [STEP2] PSKを用いた暗号化通信の実現

### SAE-2（サーバ）側の作業

In [6]:
import threading
import socket
import hashlib
from cryptography.hazmat.primitives.ciphers.aead import AESGCM

server_logs = []

def start_server_v2():
    try:
        key_bytes = hashlib.sha256(key_from_kme_2.encode('utf-8')).digest()
        aesgcm = AESGCM(key_bytes)
        
        server_sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        server_sock.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
        
        server_sock.bind(('127.0.0.1', 12346)) 
        
        server_sock.listen(1)
        server_logs.append("サーバー: 待機開始...")

        conn, addr = server_sock.accept()
        with conn:
            server_logs.append(f"サーバー: {addr} と接続しました")
            data = conn.recv(1024)
            if data:
                nonce = data[:12]
                ciphertext = data[12:]
                decrypted = aesgcm.decrypt(nonce, ciphertext, None)
                server_logs.append(f"サーバー受信完了: {decrypted.decode('utf-8')}")
        server_sock.close()
    except Exception as e:
        server_logs.append(f"サーバーエラー: {e}")

thread = threading.Thread(target=start_server_v2)
thread.start()
print("サーバー待機中... クライアントを実行してください。")

サーバー待機中... クライアントを実行してください。


### SAE-1（クライアント）側の作業

In [7]:
import socket
import os
import hashlib
from cryptography.hazmat.primitives.ciphers.aead import AESGCM

key_material = key_from_kme_1.encode('utf-8')
aesgcm = AESGCM(hashlib.sha256(key_material).digest())

print("\n--- クライアントがサーバーに暗号化して送信 ---")
print("伝えたいメッセージ: ", message := "QKDおもしろいなぁ")
nonce = os.urandom(12)
ciphertext = aesgcm.encrypt(nonce, message.encode('utf-8'), None)
print("暗号化されたメッセージ: ", ciphertext)

client_sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)

client_sock.connect(('127.0.0.1', 12346)) 

client_sock.sendall(nonce + ciphertext)
client_sock.close()
print("クライアント: 送信完了")


--- クライアントがサーバーに暗号化して送信 ---
伝えたいメッセージ:  QKDおもしろいなぁ
暗号化されたメッセージ:  b'\xe6\xf7t\x12d<I\x02\xbf\xdc\x06\x86\x8b\xd0\xacdVyv\xfa\xfb(\xe0c}\xc3\x121\xfe\xdboR\x151\r\x88HX\\5'
クライアント: 送信完了


### SAE-2（サーバ）側のログ確認

In [8]:
print("\n--- サーバーのログを表示 ---")
for log in server_logs:
    print(log)


--- サーバーのログを表示 ---
サーバー: 待機開始...
サーバー: ('127.0.0.1', 53518) と接続しました
サーバー受信完了: QKDおもしろいなぁ
